In [2]:
import pandas as pd
import os
from pathlib import Path
#from multiprocessing import Pool, Lock
#import pydicom
import glob
import random
import string
import numpy as np
import pickle
import time
import sys
import hashlib

def hash_value(value, hash_key):
    return hashlib.sha256((str(value) + hash_key).encode()).hexdigest()

In [3]:
path_to_mastertable = Path('/labs/kamaleswaranlab/MODS/Data/Emory_Data/RKENCOUNTERS_BEDLOCATION.csv')
all_meta_columns = ['AccessionNumber', 'ContentDate', 'ContentDescription', 'ContentLabel', 'ContentTime',\
                    'ImageHorizontalFlip', 'ImageRotation', 'InstanceNumber', 'IssuerOfPatientID', 'Manufacturer', \
                    'ManufacturerModelName', 'Modality', 'PatientID', 'PatientSex', 'PresentationCreationDate', \
                        'PresentationCreationTime', 'ReferringPhysicianName', 'RescaleIntercept', 'RescaleSlope',\
                        'SOPClassUID', 'SOPInstanceUID', 'SeriesDate', 'SeriesDescription', 'SeriesInstanceUID', \
                        'SeriesNumber', 'SeriesTime', 'SoftwareVersions', 'SpecificCharacterSet', 'StationName', \
                        'StudyDate', 'StudyInstanceUID', 'StudyTime', 'file', 'has_pix_array', 'category', \
                        'AcquisitionDate', 'AcquisitionDateTime', 'AcquisitionTime', 'BitsAllocated',\
                        'BitsStored', 'Columns', 'HighBit', 'ImageType', 'InstitutionName', 'LossyImageCompression',\
                    'PatientOrientation', 'PhotometricInterpretation', 'PixelRepresentation', 'ProcessingFunction',\
                    'Rows', 'SamplesPerPixel', 'StudyDescription', 'StudyID', 'AcquisitionNumber', 'ConvolutionKernel',\
                'DataCollectionDiameter', 'DeviceSerialNumber', 'DistanceSourceToDetector', 'DistanceSourceToPatient', \
                    'Exposure', 'ExposureTime', 'ImageOrientationPatient', 'ImagePositionPatient', 'InstanceCreationDate',\
                'InstanceCreationTime', 'RescaleType', 'RevolutionTime', 'RotationDirection', 'ScanOptions', 'SliceLocation', \
                    'SliceThickness', 'SpacingBetweenSlices', 'EMPI']

columns_to_deid = ['AccessionNumber',  'InstanceNumber', 'IssuerOfPatientID', \
                    'PatientID','SOPClassUID', 'SOPInstanceUID', 'StudyInstanceUID',  'InstitutionName',  'EMPI']

all_mastertable_columns = ['PAT_ID', 'EMPI_NBR', 'ENCOUNTER_NBR', 'ENCOUNTER_ID', 'BED_LOCATION_START', 'BED_LOCATION_END', 'BED_UNIT', 'BED_ROOM', 'BED_ID', 'BED_LABEL', 'HOSPITAL_SERVICE', 'ACCOMODATION_CODE', 'ACCOMODATION_DESCRIPTION']
columns_to_read = ['PAT_ID', 'EMPI_NBR', 'ENCOUNTER_NBR', 'ENCOUNTER_ID', 'BED_LOCATION_START', 'BED_LOCATION_END']

mastertable = pd.read_csv(path_to_mastertable, usecols=  columns_to_read)

path_to_combined_meta = Path('/labs/collab/Imaging/Imaging-PHI/Emory_Images/Meta/Combined_Meta.csv')
combined_meta = pd.read_csv(path_to_combined_meta)

path_to_combined_radiology_notes = Path('/labs/collab/Imaging/Imaging-PHI/Emory_Images/Meta/Combined_Radiology_Notes.csv')
radiology_notes = pd.read_csv(path_to_combined_radiology_notes)



/tmp/ipykernel_2162965/3792903567.py:24: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  mastertable = pd.read_csv(path_to_mastertable, usecols=  columns_to_read)
/tmp/ipykernel_2162965/3792903567.py:27: DtypeWarning: Columns (2,4,9,40,57,58,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_meta = pd.read_csv(path_to_combined_meta)
/tmp/ipykernel_2162965/3792903567.py:30: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  radiology_notes = pd.read_csv(path_to_combined_radiology_notes)


In [4]:
date_columns = ['ContentDate', 'PresentationCreationDate', 'SeriesDate', 'StudyDate', 'AcquisitionDate', 'InstanceCreationDate']

In [8]:
radiology_notes.head()

,ACC_NBR,PATIENT_ID,EMPI_NBR,ENCNTR_ID,HNAM_DOCUMENT_CLINICAL_ID,HNAM_DOCUMENT_CLINICAL_NM,DAY_VERIFIED,DOC_TEXT,DOC_ABSTRACT,EVENT_DOCUMENT_DESC,EVENT_DOCUMENT_KEY
0,00006DX190104831,18007243,803350,156619493.0,2228789^EVENT,XR Chest 1 View Portable,06/28/2019,REPORT\r\rEXAM: XR Chest 1 View Portable\n\nES...,REPORT\r\rEXAM: XR Chest 1 View Portable\n\nES...,00006DX20190104831,127041997.0
1,00002DX200136398,27595956,5889242,174163224.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,07/23/2020,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,00002DX20200136398,140584055.0
2,00001CT190035194,18095426,891532,149573470.0,1460399309^EVENT,CT Neuro Spine Outside Image Ref Only,02/02/2019,CT NEURO SPINE OUTSIDE IMAGE REF ONLY\r\rRefer...,CT NEURO SPINE OUTSIDE IMAGE REF ONLY\r\rRefer...,00001CT20190035194,115489211.0
3,00002DX200041549,18302477,1098583,167980557.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,02/25/2020,ADDEND\r\rThis is the incorrect report as this...,ADDEND\r\rThis is the incorrect report as this...,00002DX20200041549,135818096.0
4,00002DX200138873,19065136,1861242,174284902.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,07/27/2020,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,00002DX20200138873,140714001.0


In [16]:
df = pd.DataFrame()
df['PatientID'] = radiology_notes['EMPI_NBR'].unique()
df.to_csv('../PatientIDsWithNotes.csv', index = False)

In [13]:
mastertable.head()

,PAT_ID,EMPI_NBR,ENCOUNTER_NBR,ENCOUNTER_ID,BED_LOCATION_START,BED_LOCATION_END
0,100000034,10169234,58153720161,172113523,06/09/2020 13:57:24,06/09/2020 14:54:18
1,100000034,10169234,58153720161,172113523,06/09/2020 14:54:18,06/10/2020 00:36:04
2,100000034,10169234,58153720161,172113523,06/10/2020 00:36:04,06/10/2020 14:49:00
3,100000087,10169249,58153788093,136538079,04/03/2018 13:24:01,04/03/2018 14:01:27
4,100000087,10169249,58153788093,136538079,04/03/2018 14:01:27,04/03/2018 17:00:40


In [36]:
encounter_table = mastertable[['ENCOUNTER_ID', 'ENCOUNTER_NBR']].drop_duplicates()

In [32]:
radiology_notes = radiology_notes.loc[radiology_notes.ENCNTR_ID.notna()]

In [33]:
radiology_notes.ENCNTR_ID = radiology_notes.ENCNTR_ID.astype('int64')

In [37]:
df = pd.merge(radiology_notes, encounter_table, 'inner', left_on = 'ENCNTR_ID', right_on = 'ENCOUNTER_ID')

In [39]:
df.to_csv('/labs/collab/Imaging/Imaging-PHI/Emory_Images/Meta/Combined_Radiology_Notes_with_EncounterNumber.csv')

In [10]:
combined_meta['AccessionNumber'].values[500]

'00005DX150059990'

In [17]:
row = combined_meta.iloc[100]
accession_number = row['AccessionNumber']
radiology_note = radiology_notes.loc[radiology_notes['EVENT_DOCUMENT_DESC'] == accession_number]
if len(radiology_note) == 0:
    print("No radiology note for accession number: ", accession_number)

#Append radiology note columns to combined_meta
for col in radiology_note.columns:
    combined_meta.at[idx, col] = radiology_note[col].values[0]

No radiology note for accession number:  00003DX170043130


IndexError: index 0 is out of bounds for axis 0 with size 0

In [22]:
acc = radiology_notes['ACC_NBR']
acc_comb = combined_meta['AccessionNumber']

In [23]:
common = set(acc).intersection(set(acc_comb))
len(common)

81819

In [25]:
81819/len(acc)

0.016045297632926874

In [15]:
radiology_notes.head()

,ACC_NBR,PATIENT_ID,EMPI_NBR,ENCNTR_ID,HNAM_DOCUMENT_CLINICAL_ID,HNAM_DOCUMENT_CLINICAL_NM,DAY_VERIFIED,DOC_TEXT,DOC_ABSTRACT,EVENT_DOCUMENT_DESC,EVENT_DOCUMENT_KEY
0,00006DX190104831,18007243,803350,156619493.0,2228789^EVENT,XR Chest 1 View Portable,06/28/2019,REPORT\r\rEXAM: XR Chest 1 View Portable\n\nES...,REPORT\r\rEXAM: XR Chest 1 View Portable\n\nES...,00006DX20190104831,127041997.0
1,00002DX200136398,27595956,5889242,174163224.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,07/23/2020,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,00002DX20200136398,140584055.0
2,00001CT190035194,18095426,891532,149573470.0,1460399309^EVENT,CT Neuro Spine Outside Image Ref Only,02/02/2019,CT NEURO SPINE OUTSIDE IMAGE REF ONLY\r\rRefer...,CT NEURO SPINE OUTSIDE IMAGE REF ONLY\r\rRefer...,00001CT20190035194,115489211.0
3,00002DX200041549,18302477,1098583,167980557.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,02/25/2020,ADDEND\r\rThis is the incorrect report as this...,ADDEND\r\rThis is the incorrect report as this...,00002DX20200041549,135818096.0
4,00002DX200138873,19065136,1861242,174284902.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,07/27/2020,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,00002DX20200138873,140714001.0


In [ ]:
for idx, row in combined_meta.iterrows():
    accession_number = row['AccessionNumber']
    radiology_note = radiology_notes.loc[radiology_notes['AccessionNumber'] == accession_number]
    if len(radiology_note) == 0:
        print("No radiology note for accession number: ", accession_number)
        continue

    #Append radiology note columns to combined_meta
    for col in radiology_note.columns:
        combined_meta.at[idx, col] = radiology_note[col].values[0]
    
path_to_combined_meta = Path('/labs/collab/Imaging/Imaging-PHI/Emory_Images/Meta/Combined_Meta_' + year + '_Radiology.pkl')
combined_meta.to_pickle(path_to_combined_meta)